# CDS services in jupyter notebooks
Welcome to a Jupyter notebook demonstrating how to access some CDS services with Python. Jupyter notebooks offer an interactive way to use Python. If you are viewing this notebook with Binder you can execute every cell of code either by pushing the little 'play' <img src="images/jupyter_run-cell.png" alt="" style="width:70px; display: inline-block;"/> symbol on the left of the cell (appears when the mouse pointer moves in its vincitity) or by selecting a cell and pressing 'Shift' + 'Enter' at the same time. Note that cells that have not yet been executed have no number assigned to them, later on they are assigned the number of the order in which they were executed. If they are run again, they get a new number. 

The aim of this tutorial is to open a Aladin Lite widget, find and download a catalogue from Vizier and overlay the sources of this catalogue on the sky view in the Aladin Lite widget. 

Python draws its power from the large variety of packages that are available for everyone to install and use. For this tutorial we will be using two packages: `ipyaladin` (https://github.com/cds-astro/ipyaladin, allows to load Aladin Lite in a jupyter notebook) and `Vizier` from the `astroquery` package (https://astroquery.readthedocs.io/en/latest/index.html#introduction, allows to query VizieR from this notebook). Let's import these packages by running the following cell:

In [ ]:
from astroquery.vizier import Vizier
import ipyaladin.aladin_widget as ipyal
import pyvo

print('Hello there, the packages are imported and we are ready to go :)')


## Start Aladin Lite
We start this tutorial by initialising Aladin Lite. In the first line of code we tell Python that the variable `aladin` is now assigned to an Aladin Lite widget. This widget is supposed to **center** on the Antennae galaxies, set a **field of view** of 0.7deg and show colour images from the DSS **survey**. Typing `aladin` in the second line of code, tells the notebook to display the widget. 

In [ ]:
aladin = ipyal.Aladin(target='Antennae', fov=0.7, survey='P/DSS2/color')
aladin


As with any Aladin Lite implementation, you can interact with this widget: 
 - to zoom in and out place you mouse pointer on top of the image and scroll. 
 - with <img src="images/ipyaladin_layer.png" alt="the Layer Button" style="width:30px; display: inline-block;"/>  you can select other image surveys and manage the current view.
 - if you like to look at another target, you can use the search field <img src="images/ipyaladin_search.png" alt="the Search Button" style="width:30px; display: inline-block;"/> to get there. 

These interactions can also be steered by changing properties of the variable `aladin`. If for example you wanted to focus on M101 instead of the Antennae galaxies, do:

In [ ]:
aladin.target = 'M101'


## Find and download tables from VizieR
Besides using Aladin we can also query VizieR from within this Jupyter notebook. To do so we use the `Vizier` subpackage from the `astroquery` package. As we remember from before, the catalogue of peculiar galaxies was called "Arp's peculiar galaxies" and was published by Webb. Hence, we ask `Vizier` to find all catalogues that have a match with the keywords 'Arp Webb' and write the result into the variable `catalog_list_arp` (first line of code). Then we tell Python to `print` out the query result in a readable way (second line of code).  

In [ ]:
catalog_list_arp = Vizier.find_catalogs('Arp')
for k, v in catalog_list_arp.items():
    print(k, ': ', v.description)


Alternatively you can also search for the table with `pyvo` using the Table Access Protocol (TAP) to query VizieR. In this case, you first need to define the TAP endpoint of VizieR and then send a query written in ADQL (an SQL-like query language optimised for astronomy) to this endpoint. Finally we again print out the result of this query.

In [ ]:
tap_vizier = pyvo.dal.TAPService(
    'https://tapvizier.cds.unistra.fr/TAPVizieR/tap')
query = """SELECT  *  FROM tap_schema.tables 
               WHERE table_name LIKE '%arp%' """
catalog_list_arp = tap_vizier.search(query).to_table()
catalog_list_arp['table_name', 'description']


The catalogue that we are interested in today is "VII/192" and its description is "Arp's Peculiar Galaxies (Webb 1996)". In the TAP query we already see that there are two tables in this catalogue: `VII/192/arplist` and `VII/192/arpord`. To be able to work with these data, let's load them into this notebook. 

Again we can use either `astroquery` or `pyvo`. Which one you chose is up to you and which one you like better. Attention **Spoiler:** over the week you'll learn more about TAP and ADQL.

In oder to get the full catalogue with `astroquery`, we first set the row limit to infinite (i.e. `-1` in the notation of this package) and then ask `Vizier` to write the content of the catalogue into the variable `catalogs_arp` (second line):

In [ ]:
Vizier.ROW_LIMIT = -1
catalogs_arp = Vizier.get_catalogs('VII/192')


Now let's inspect, what we got:

In [ ]:
print(catalogs_arp)


As we have seen in the first part of the tutorial, the catalogue "Arp's Peculiar Galaxies (Webb 1996)" comes with two tables: arpord and arplist. As you can see from the printout `Vizier` has downloaded both of them. However, we are still only interested in the arplist (index `1` in the TableList). Therefore, we make Python write only the arplist table into a new variable `table_arplist` (first line) and then display the table (second line):

In [ ]:
table_arplist = catalogs_arp[1]
table_arplist


Nice, we got the data we want. Before we move on, let's have a look at how we can use `pyVO` to get the same table. Just for safety we define again the VizieR TAP endpoint. It is the same one as above, but if the cells of this notebook are not run one after another, this safety measure ensure we always query the right service ;) 

The we write a simple query telling VizieR that we want all the rows and all the columns from the table `VII/192/arplist`. Note that we have to encompass the table name with quotation marks due to the special character `/` in the table name. 

In [ ]:
query = """SELECT  *  FROM "VII/192/arplist" """
table_arplist = tap_vizier.search(query).to_table()
table_arplist


There we go, we now know two nifty ways to get VizieR tables with Python. Obviously there are many other things you can do with this tools. For example if you set your TAP endpoint to `http://simbad.u-strasbg.fr:80/simbad/sim-tap` instead of the VizieR enpoint, you can query SIMBAD just like that. 

Note that not all rows but only the first and last 10 rows for all columns are shown in the display. To finish off this little excursion, we now want to visualise the location of the entries of this table in our Aladin Lite widget. To do so, we tell Python to take the variable `aladin` (remember that this was the name of the widget) and add the table `table_arplist` to it:

In [ ]:
aladin.add_table(table_arplist)


Now scroll back up to the Aladin Lite widget. You will find that the location of the sources in the arplist table are marked with coloured symbols. Again you can zoom in and out to look at the different sources. If you click on one of the colour symbols, you will be able to see the corresponding row in the bottom of the Aladin Lite widget. 